This project will mainly be used for the capstone project.

In [3]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## This is the assignment for: Segmenting and Clustering Neighborhoods in Toronto


### Part 1

In [34]:
import requests
from bs4 import BeautifulSoup

In [53]:

!pip install shapely

     |████████████████████████████████| 1.8MB 13.9MB/s eta 0:00:01


In [54]:

!pip install geopandas

     |████████████████████████████████| 931kB 12.5MB/s eta 0:00:01
     |████████████████████████████████| 10.4MB 21.2MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 29.8MB/s eta 0:00:01


In [56]:
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns

In [57]:

#Getting the data from url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
data = pd.read_json(df[0].to_json(orient='records'))

In [58]:
data.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


In [61]:
data_new = data[data['Borough'] != 'Not assigned']
data_new.head()

,Borough,Neighbourhood,Postcode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,North York,Lawrence Heights,M6A
6,North York,Lawrence Manor,M6A


In [62]:
#Grouping Data
data_new= data_new.groupby(['Borough', 'Postcode'], as_index=False).agg(','.join)

In [63]:
data_new['Neighbourhood'] = np.where(data_new['Neighbourhood'] == 'Not assigned', data_new['Borough'], data_new['Neighbourhood'])

In [65]:
data_new.shape

(103, 3)

## Now I will add the geospatial data

In [68]:
url = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url)
df_geo.rename(columns={'Postal Code':'Postacode'}, inplace=True)
df_geo.head()

,Postacode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [69]:

df_canada_combined = pd.merge(data_new, df_geo, how='left', on='Postacode')
df_canada_combined.head(12)

KeyError: 'Postacode'